In [ ]:
!pip install langchain-openai langgraph langchain-experimental gradio python-dotenv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found exis

In [3]:
import os
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import Tool, AgentExecutor, create_openai_tools_agent
from langchain_experimental.tools import PythonREPLTool
from langgraph.graph import END, StateGraph
from langchain_core.messages import AIMessage, HumanMessage, AnyMessage
from typing_extensions import TypedDict
from typing import Annotated
import operator

### ====== AGENT SETUP ======

# Create developer agent
def create_developer_agent(llm: ChatOpenAI, tools: list, system_prompt: str, verbose=False):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=verbose)
    return executor

# Create tester agent
def create_tester_agent(llm: ChatOpenAI, prompt: str):
    tester_prompt = ChatPromptTemplate.from_messages([
        ("system", prompt),
        MessagesPlaceholder(variable_name="messages"),
    ])
    return tester_prompt | llm

### ====== DEVELOPER NODE ======
def developer_node(state):
    developer_prompt = """You are an intelligent coder trained in writing Python code.
    Your job is to write and iteratively refine code based on feedback and test results.
    At every step, output the COMPLETE revised version. Focus on quality, comments, and edge cases."""

    tools = [PythonREPLTool()]
    dev_agent = create_developer_agent(state['llm'], tools, developer_prompt, verbose=False)

    response = dev_agent.invoke({"messages": state['conversation_history']})
    state['conversation_history'].append(AIMessage(content=response['output']))
    state['final_code'] = response['output']

    return state

### ====== TESTER NODE ======
def tester_node(state):
    tester_prompt = """You are a highly skilled tester. Write exhaustive unit tests for the code and give feedback.
    Include a score (0-10) for quality and percentage of tests passed. Suggest fixes where necessary.
    If tests fail, generate critique and recommendations."""

    tester_agent = create_tester_agent(state['llm'], tester_prompt)
    response = tester_agent.invoke({"messages": state['conversation_history']})

    state['conversation_history'].append(HumanMessage(content=response.content))
    state['reflection_count'] += 1
    return state

### ====== STATE TYPE ======
class GraphState(TypedDict):
    llm: ChatOpenAI
    max_reflections: int
    reflection_count: Annotated[int, operator.add]
    conversation_history: Annotated[list[AnyMessage], operator.add]
    final_code: str

### ====== GRAPH BUILDING ======
def build_graph():
    builder = StateGraph(GraphState)

    builder.add_node("developer_node", developer_node)
    builder.add_node("tester_node", tester_node)

    builder.set_entry_point("developer_node")

    def should_continue(state):
        return END if state["reflection_count"] >= state["max_reflections"] else "tester_node"

    builder.add_conditional_edges("developer_node", should_continue, ["tester_node", END])
    builder.add_edge("tester_node", "developer_node")

    return builder.compile()

### ====== MAIN REFLECTION RUN ======
def run_reflection(api_key, prompt, max_reflections):
    os.environ["OPENAI_API_KEY"] = api_key
    llm = ChatOpenAI(model="gpt-4o", temperature=0)

    graph = build_graph()
    inputs = {
        "llm": llm,
        "max_reflections": max_reflections,
        "reflection_count": 0,
        "conversation_history": [HumanMessage(content=prompt)],
        "final_code": ""
    }

    for _ in graph.stream(inputs, stream_mode="values"):
        pass  # It mutates state internally

    final_code = inputs["final_code"]
    full_conversation = "\n\n".join([f"{type(msg).__name__.replace('Message', '')}: {msg.content}" for msg in inputs["conversation_history"]])

    return final_code.strip(), full_conversation.strip()

### ====== GRADIO UI ======

with gr.Blocks() as demo:
    gr.Markdown("# 🤖 Developer vs Tester – Reflection Agent")
    gr.Markdown("Enter your OpenAI API key and prompt. Agents will iteratively refine the code and output their conversation.")

    api_key_input = gr.Textbox(label="🔐 OpenAI API Key", type="password")
    prompt_input = gr.Textbox(label="💡 Your Prompt", placeholder="e.g., Write a Python function to sort a list of dictionaries by key")
    reflection_slider = gr.Slider(label="🔁 Max Reflections", minimum=1, maximum=5, step=1, value=3)

    run_btn = gr.Button("🚀 Run Agent Simulation")

    code_output = gr.Code(label="🧾 Final Generated Code", language="python")
    convo_output = gr.Textbox(label="📜 Full Developer–Tester Conversation", lines=20)

    run_btn.click(fn=run_reflection, inputs=[api_key_input, prompt_input, reflection_slider], outputs=[code_output, convo_output])

if __name__ == "__main__":
    demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ade852361a353873ed.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
